In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
from splinter import Browser
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Path to chromedriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\rak_t\.wdm\drivers\chromedriver\win32\91.0.4472.19\chromedriver.exe] found in cache


## Visit the NASA mars news site

In [3]:
# Scrape NASA Mars News site for News title and paragraph text
url = 'https://mars.nasa.gov/news/'
#response = requests.get(
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8959/nasas-insight-mars-lander-gets-a-power-boost/" target="_self">NASA's InSight Mars Lander Gets a Power Boost</a></div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's InSight Mars Lander Gets a Power Boost"

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.'

## JPL Space Images Featured Image

In [8]:
# Scrape URL featured space images. Ues splinter to find current feature Marks full-size image and assign to variable
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [11]:
html= browser.html
im_soup=soup(html, 'html.parser')
featured_image= im_soup.select('li.slide a.fancybox')
#make a list
img_list = [i.get('data-fancybox-href') for i in featured_image]

#combine the base url with the first img url
base_url = 'https://data-class-jpl-space.s3.amazonaws.com'
featured_image_url = base_url + img_list[1]

print(featured_image_url)

IndexError: list index out of range

In [12]:
# Find and click the full image button
#browser.find_by_css('.btn')

full_image_elem = browser.find_by_css('.btn')
full_image_elem.click()


In [ ]:
# Find the more info button and click that
#parse the source
#use splinter then use bs to parse through (take current page pass into bs to then parse) (look at last 3 cells)
image/featured/mars1.jpg, 

more_info_button = browser.links.find_by_partial_text('more info')
more_info_button.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

In [ ]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

## Mars Facts

In [ ]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.head()

In [ ]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

In [ ]:
df.to_html()

## Hemispheres

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

In [ ]:
hemisphere_image_urls

In [ ]:
browser.quit()